# ECE1508: Applied Deep Learning
# Assignment 1
## Lastname, Firstname

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt

Try an example

In [ ]:
np.ones(3)

## Programming Question 1

In [8]:
class LinearMachine():
    def __init__(self):
        # Define weights as attribute
        # Set the initial values randomly

        # ============ COMPLETE HERE ================
        # self._weights = ## COMPLETE ##
        # ============================================
        pass

    def data_synthesizer(self, dataset_size, velocity_mean, velocity_var,
                         height_mean, height_var):

        # Draw random velocity and height
        # Let the array size to be <dataset_size>
        # Don't forget to make them positive using np.abs()
        
        # ============ COMPLETE HERE ================
        # v_sample = ## COMPLETE ##
        # h_sample = ## COMPLETE ##

        # Compute distance d via Newton's law
        # d_sample = ## COMPLETE ##
        # ============================================

        # Make the dataset
        # ============ COMPLETE HERE ================
        # dataset = ## COMPLETE ##
        # return dataset
        # ============================================
        pass

    def train_GD(self, dataset, lr, delta = float("inf"), max_iterations = 1e12):

        # We can reset the weights to some initial random values
        # ============ COMPLETE HERE ================
        # self._weights = ## COMPLETE ##
        # ============================================

        # read data-points and labels
        # ============ COMPLETE HERE ================
        # x_mat = ## COMPLETE ##
        # d_mat = ## COMPLETE ##
        # ============================================

        # Count number of iterations to stop after max iterations
        counter = 0

        # Initiate risk
        risk = 0

        while delta > 0 and counter < max_iterations:
            
            # Compute Gradient
            # ============ COMPLETE HERE ================
            # grad = ## COMPLETE ## Multiple lines
            # ============================================

            # Save risk at current weights
            # ============ COMPLETE HERE ================
            old_risk = 0 ## COMPLETE ##
            # ============================================

            # Update weights using GD with learning rate lr
            # ============ COMPLETE HERE ================
            # self._weights = ## COMPLETE ##
            # ============================================

            # Compute new risk
            # ============ COMPLETE HERE ================
            # risk = ## COMPLETE ## Multiple lines
            # ============================================

            # Update stopping criteria
            delta = np.abs(risk - old_risk)
            counter += 1

        # Print out if the code stops due to max iterations, i.e., GD did not converge
        if counter == max_iterations:
            print(f"GD dos not converge: train_GD hit max iterations = {max_iterations}!")

        return risk
    

    def train(self, dataset):
          # Find the minimizer
          # ============ COMPLETE HERE ================
          # self._weights = ## COMPLETE ## Multiple lines
          # ============================================


          # Compute minimum risk
          # ============ COMPLETE HERE ================
          risk_opt = 0  ## COMPLETE ## Multiple lines
          # ============================================
          return risk_opt

    def test(self, J, velocity_mean, velocity_var, height_mean, height_var):
          
          # Generate an independent dataset for test
          # ============ COMPLETE HERE ================
          dataset = [] ## COMPLETE ##
          # ============================================

          # Compute the risk via the trained weights
          # ============ COMPLETE HERE ================
          # ## COMPLETE ## Multiple lines
          # ============================================

          # Compute test risk
          # ============ COMPLETE HERE ================
          risk_test = 0  ## COMPLETE ##
          # ============================================
          return risk_test

### Gradient Descent vs Optimal Training

In [ ]:
def Optimality_Gap(lr_list):
    myModel = LinearMachine()
    dataset = myModel.data_synthesizer(100, 1, 5, 3, 3)

    # Initiate optimality gap vector
    Opt_gap = []

    # Find optimal weights
    myModel.train(dataset)
    weights_optimal = myModel._weights

    # Find GD trained weights 
    for lr in lr_list:
        # ============ COMPLETE HERE ================
        weights_GD = 0 ## COMPLETE ## Multiple lines
        # ============================================

        # Append the optimality gap 
        Opt_gap.append(np.linalg.norm(weights_optimal-weights_GD))


    # Plot 
    plt.figure()
    plt.plot(lr_list, Opt_gap)
    plt.title('GD vs Optimal Training')
    plt.xlabel('Learning Rate')
    plt.ylabel('Distance between Optimal and GD Weights')

    plt.show()


# Execute your implementation
Optimality_Gap([0.1, 0.01, 0.001, 0.0001])

### Test Loop

In [ ]:
def eval_model(T, I, J, velocity_mean, velocity_var, height_mean, height_var):
  myModel = LinearMachine()
  lr = 0.001
  
  # Initiate the risk 
  emp_risk = 0

  # Loop over t
  for t in range(T):
    # ============ COMPLETE HERE ================
    emp_risk = 0 # # ## COMPLETE ## Multiple lines
    # # ============================================
  
  # average rist over datasets
  emp_risk = emp_risk / T

  return emp_risk


# Make list for plot
emp_risk_list = []
T, J = 100, 10

# loop over I
I_list = [10*i for i in range(1,21)]
for I in I_list:
  emp_risk_list.append(eval_model(T, I, J, 1, 5, 3, 3))


# Plot
plt.figure()
plt.plot(I_list, emp_risk_list)
plt.title('Test Risk against Size of Dataset')
plt.xlabel('Dataset Size')
plt.ylabel('Test Risk')


## Programming Question 2

### Perceptron Machine

In [30]:
class PerceptronMachine():
    def __init__(self):
        # define weights as attribute and initiate randomly
        self._dimensions = 9
        # ============ COMPLETE HERE ================
        # ## COMPLETE ## Multiple lines
        # ============================================


    def forward(self, datapoint):
        # Affine function
        # ============ COMPLETE HERE ================
        # ## COMPLETE ## Multiple lines
        # ============================================

        # Activate by step function
        return 1 if affine > 0 else 0

    def train(self, dataset, lr):
        # initiate weights again randomly as we start with training
        # ============ COMPLETE HERE ================
        # ## COMPLETE ## Multiple lines
        # ============================================

        # initiate error_flag with 1 to get to the while loop
        error_flag = 1

        while error_flag > 0:
            # We initially have no error
            error_flag = 0
            for data in dataset:
                # Read data
                x = data[0]
                label = data[1]

                # Pass it forward
                y = self.forward(x)

                # Implement inner loop
                if y != label:
                    # Find sign(z) 
                    # ============ COMPLETE HERE ================
                    # ## COMPLETE ## 
                    # ============================================

                    # Set movement vector = sgn(z) * x
                    # ============ COMPLETE HERE ================
                    # ## COMPLETE ## 
                    # ============================================

                    # Update
                    # ============ COMPLETE HERE ================
                    # ## COMPLETE ## 
                    # ============================================

                    # Since we had error, we set the flag 1 again
                    error_flag = 1


### X Pattern

In [ ]:
# Generating the X Pattern
# Run this code to see X pattern

x = 256* np.ones([3,3],dtype = int)

for i in range(3):
    x[i,i] = 0
    x[i,2-i] = 0

plt.imshow(x,cmap='gray')

### Generate Dataset and Train

In [ ]:
# Generate the Dataset
# Initiate empty dataset
dataset = []
image_list = []

# We have 2^9=512 possibilities: loop over them
for i in range(512): 
    # generate 9-dimensional vector
    # ============ COMPLETE HERE ================
    # ## COMPLETE ## Multiple lines
    # ============================================

    # label the vector
    # ============ COMPLETE HERE ================
    # ## COMPLETE ## Multiple lines
    # ============================================


# Instantiate a perceptron and train it 
myModel = PerceptronMachine()
myModel.train(dataset, 0.001)

### Test Trained Model

In [ ]:
# Initiate the test set
testset = []

# Add the X pattern
testset.append([0, 1, 0, 1, 0, 1, 0, 1, 0])

# Choose some data samples
for i in range(10):
    rand_idx = int(np.random.rand() * 512) % 512
    while rand_idx == 170:
        rand_idx = int(np.random.rand() * 512) % 512
    testset.append(dataset[rand_idx][0])


# Test the set: only the first one should return 1
for test in testset:
    print(f"Is it X patter? Answer: {myModel.forward(test)}")
